In [1]:
NREL_API_KEY = None  # <-- please set your NREL API key here
# note you must use "quotes" around your key as it is a string.

if NREL_API_KEY is None:
       NREL_API_KEY = 'DEMO_KEY'  # OK for this demo, but better to get your own key

In [2]:
import pvlib

metdata, metadata = pvlib.iotools.get_psm3(
    latitude=18.4671, longitude=-66.1185,
    api_key=NREL_API_KEY,
    email='silvana.ovaitt@nrel.gov',  # <-- any email works here fine
    names='tmy', map_variables=True)
metadata

{'Source': 'NSRDB',
 'Location ID': '1493571',
 'City': '-',
 'State': '-',
 'Country': '-',
 'Time Zone': -4,
 'Local Time Zone': -4,
 'Dew Point Units': 'c',
 'DHI Units': 'w/m2',
 'DNI Units': 'w/m2',
 'GHI Units': 'w/m2',
 'Temperature Units': 'c',
 'Pressure Units': 'mbar',
 'Wind Direction Units': 'Degrees',
 'Wind Speed Units': 'm/s',
 'Surface Albedo Units': 'N/A',
 'Version': '3.2.0',
 'latitude': 18.45,
 'longitude': -66.1,
 'altitude': 6}

## 2. Modeling with bifacial_radiance

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import bifacial_radiance as br

In [4]:
br.__version__

'0.4.2+239.g1e65c8a.dirty'

In [5]:
import os
from pathlib import Path

testfolder = 'TEMP'

if not os.path.exists(testfolder):
    os.makedirs(testfolder)
    
print ("Your simulation will be stored in %s" % testfolder)

Your simulation will be stored in TEMP


In [6]:
radObj = br.RadianceObj('Sim3',path=testfolder)

path = TEMP


In [7]:
# Some of the names changed internally. While bifacial_radiance updates their expected names, we are renaming the values here
metadata['timezone'] = metadata['Time Zone']
metadata['county'] = '-'
metadata['elevation'] = metadata['altitude']
metadata['state'] = metadata['State']
metadata['country'] = metadata['Country']
metdata['Albedo'] = metdata['albedo']

Use NSRDBWeatherData to enter data the downloaded data in dataframe and dictionary forma for meteorological data and metadata respectively

In [8]:
#starttime can be 'MM_DD', or 'MM_DD_HH'
metData = radObj.NSRDBWeatherData(metadata, metdata, starttime='11_08_09', endtime='11_08_11',coerce_year=2021)

8760 line in WeatherFile. Assuming this is a standard hourly WeatherFile for the year for purposes of saving Gencumulativesky temporary weather files in EPW folder.
Coercing year to 2021
Filtering dates
Saving file EPWs\metdata_temp.csv, # points: 8760
Calculating Sun position for center labeled data, at exact timestamp in input Weather File


In [9]:
metData.datetime  # printing the contents of metData to see how many times got loaded.

[Timestamp('2021-11-08 09:30:00-0400', tz='pytz.FixedOffset(-240)'),
 Timestamp('2021-11-08 10:30:00-0400', tz='pytz.FixedOffset(-240)')]

In [10]:
# -- establish tracking angles
hub_height = 1.5
pitch = 5
sazm = 180  # Tracker axis azimuth
modulename = 'PVmodule'
fixed_tilt_angle = None
gcr = 2 / pitch


trackerParams = {'limit_angle':50,
             'angledelta':5,
             'backtrack':True,
             'gcr':gcr,
             'cumulativesky':False,
             'azimuth': sazm,
             'fixed_tilt_angle': fixed_tilt_angle
             }

In [11]:
trackerdict = radObj.set1axis(**trackerParams)

In [12]:
radObj.setGround(0.2) 

Loading albedo, 1 value(s), 0.200 avg
1 nonzero albedo values.


In [13]:
radObj.gendaylit1axis()

Creating ~2 skyfiles. 
Created 2 skyfiles in /skies/


{'2021-11-08_0930': {'surf_azm': 90.0,
  'surf_tilt': 45.01,
  'theta': -45.01,
  'dni': 793.0,
  'ghi': 583.0,
  'dhi': 98.0,
  'temp_air': 28.0,
  'wind_speed': 1.1,
  'skyfile': 'skies\\sky2_18.45_-66.1_2021-11-08_0930.rad'},
 '2021-11-08_1030': {'surf_azm': 90.0,
  'surf_tilt': 28.41,
  'theta': -28.41,
  'dni': 850.0,
  'ghi': 730.0,
  'dhi': 103.0,
  'temp_air': 29.0,
  'wind_speed': 0.9,
  'skyfile': 'skies\\sky2_18.45_-66.1_2021-11-08_1030.rad'}}

In [14]:
module=radObj.makeModule(name=modulename, x=1,y=2)


Module Name: PVmodule
Module PVmodule updated in module.json
Pre-existing .rad file objects\PVmodule.rad will be overwritten



In [15]:
sceneDict = {'pitch':pitch, 
             'hub_height': hub_height,
             'nMods': 5,
             'nRows': 2,
             'tilt': fixed_tilt_angle,  
             'sazm': sazm
             }

In [16]:
trackerdict = radObj.makeScene1axis(module=modulename,sceneDict=sceneDict)



Making ~2 .rad files for gendaylit 1-axis workflow (this takes a minute..)
2 Radfiles created in /objects/


In [17]:
trackerdict = radObj.makeOct1axis()


Making 2 octfiles in root directory.
Created 1axis_2021-11-08_0930.oct
Created 1axis_2021-11-08_1030.oct


In [18]:
trackerdict = radObj.analysis1axis(customname = 'Module', 
                                   sensorsy=2, modWanted=2,
                                   rowWanted=1)

Linescan in process: 1axis_2021-11-08_0930Module_Row1_Module2_Front
Linescan in process: 1axis_2021-11-08_0930Module_Row1_Module2_Back
Saved: results\irr_1axis_2021-11-08_0930Module_Row1_Module2.csv
Index: 2021-11-08_0930. Wm2Front: 821.8948. Wm2Back: 74.16203999999999
Linescan in process: 1axis_2021-11-08_1030Module_Row1_Module2_Front
Linescan in process: 1axis_2021-11-08_1030Module_Row1_Module2_Back
Saved: results\irr_1axis_2021-11-08_1030Module_Row1_Module2.csv
Index: 2021-11-08_1030. Wm2Front: 831.2946000000001. Wm2Back: 92.94554500000001


In [19]:
trackerdict = radObj.calculateResults(bifacialityfactor=0.7, agriPV=False)

No CECModule data passed; using default for Prism Solar BHC72-400


c:\users\sayala\documents\github\bifacial_radiance\bifacial_radiance\performance.py:64: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  alpha_sc=float(CECMod.alpha_sc),
c:\users\sayala\documents\github\bifacial_radiance\bifacial_radiance\performance.py:65: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  a_ref=float(CECMod.a_ref),
c:\users\sayala\documents\github\bifacial_radiance\bifacial_radiance\performance.py:66: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  I_L_ref=float(CECMod.I_L_ref),
c:\users\sayala\documents\github\bifacial_radiance\bifacial_radiance\performance.py:67: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(

In [21]:
ResultPVWm2Back = radObj.CompiledResults.iloc[0]['Grear_mean']
ResultPVWm2Front = radObj.CompiledResults.iloc[0]['Gfront_mean']
ResultPVWm2Back

74.16203999999999

In [26]:
radObj.CompiledResults

,timestamp,module,row,POA_eff,Grear_mean,Gfront_mean,Pout_raw,Pout_Gfront,BGG,BGE,Mismatch,Pout,Wind Speed,DNI,DHI,GHI
0,2021-11-08_0930,2,1,873.808228,74.162040,821.8948,314.050775,297.462232,6.316311,5.576689,0.000042,314.050643,1.1,793.0,98.0,583.0
1,2021-11-08_1030,2,1,896.356482,92.945545,831.2946,319.367970,298.857203,7.826573,6.863066,0.000017,319.367915,0.9,850.0,103.0,730.0


In [23]:
resolutionGround = 1  #meter. use 1 for faster test runs
numsensors = int((pitch/resolutionGround)+1)
modscanback = {'xstart': 0, 
                'zstart': 0.05,
                'xinc': resolutionGround,
                'zinc': 0,
                'Ny':numsensors,
                'orient':'0 0 -1'}

# Analysis for GROUND
trackerdict = radObj.analysis1axis(customname = 'Ground',
                                   modscanfront=modscanback, sensorsy=1)

Linescan in process: 1axis_2021-11-08_0930Ground_Row1_Module3_Front
Linescan in process: 1axis_2021-11-08_0930Ground_Row1_Module3_Back
Saved: results\irr_1axis_2021-11-08_0930Ground_Row1_Module3_Front.csv
Saved: results\irr_1axis_2021-11-08_0930Ground_Row1_Module3_Back.csv
Index: 2021-11-08_0930. Wm2Front: 570.3255499999999. Wm2Back: 63.12785
Linescan in process: 1axis_2021-11-08_1030Ground_Row1_Module3_Front
Linescan in process: 1axis_2021-11-08_1030Ground_Row1_Module3_Back
Saved: results\irr_1axis_2021-11-08_1030Ground_Row1_Module3_Front.csv
Saved: results\irr_1axis_2021-11-08_1030Ground_Row1_Module3_Back.csv
Index: 2021-11-08_1030. Wm2Front: 609.4579166666667. Wm2Back: 77.27292


In [41]:
keys=list(trackerdict.keys())

groundIrrad = []
moduleGfront = []
moduleGrear = []

for key in keys:
    groundIrrad.append(c]['Results'][1]['Wm2Front'])
    moduleGfront.append(trackerdict[keys[key]]['Results'][0]['Wm2Front'])
    moduleGrear.append(trackerdict[keys[key]]['Results'][0]['Wm2Back'])
    

In [58]:
trackerdict[keys[key]]['Results'][1]['Wm2Front']

[538.7146, 573.0331, 577.3693, 576.9442, 577.7593, 578.1328]

In [50]:
trackerdict[keys[key]]
trackerdict[keys[key]]['dni']
trackerdict[keys[key]]['ghi']
trackerdict[keys[key]]['dhi']
trackerdict[keys[key]]['temp_air']
trackerdict[keys[key]]['wind_speed']
trackerdict[keys[key]]['scene'].sceneDict['pitch']
trackerdict[keys[]]
trackerdict[]

KeyError: 'sceneDict'

5

In [ ]:
moduleGfront = trackerdict[keys[key]]['Results'][0]['Wm2Front']
moduleGrear = trackerdict[keys[key]]['Results'][0]['Wm2Back']